# im2latex(S): Deep Learning Model

&copy; Copyright 2017 Sumeet S Singh

    This file is part of the im2latex solution (by Sumeet S Singh in particular since there are other solutions out there).

    This program is free software: you can redistribute it and/or modify
    it under the terms of the Affero GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    Affero GNU General Public License for more details.

    You should have received a copy of the Affero GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

## The Model
* Follows the [Show, Attend and Tell paper](https://www.semanticscholar.org/paper/Show-Attend-and-Tell-Neural-Image-Caption-Generati-Xu-Ba/146f6f6ed688c905fb6e346ad02332efd5464616)
* [VGG ConvNet (16 or 19)](http://www.robots.ox.ac.uk/~vgg/research/very_deep/) without the top-3 layers
    * Pre-initialized with the VGG weights but allowed to train
    * The ConvNet outputs $D$ dimensional vectors in a WxH grid where W and H are 1/16th of the input image size (due to 4 max-pool layers). Defining $W.H \equiv L$ the ConvNet output represents L locations of the image $i \in [1,L]$ and correspondingly outputs to L annotation vectors $a_i$, each of size $D$.
* A dense (FC) attention model: The deterministic soft-attention model of the paper computes $\alpha_{t,i}$ which is used to select or blend the $a_i$ vectors before being fed as inputs to the decoder LSTM network (see below).
    * Inputs to the attention model are $a_i$ and $h_{t-1}$ (previous hidden state of LSTM network - see below)
    and $$\alpha_{t,i} = softmax ( f_{att}(a_i, h_{t-1}) )$$
* A Decoder model: A conditioned LSTM that outputs probabilities of the text tokens $y_t$ at each step. The LSTM is conditioned upon $z_t = \sum_i^L(\alpha_{t,i}.a_i)$ and takes the previous hidden state $h_{t-1}$ as input. In addition, an embedding of the previous output $Ey_{t-1}$ is also input to the LSTM. At training time, $y_{t-1}$ would be derived from the training samples, while at inferencing time it would be fed-back from the previous predicted word.
    * $y$ is taken from a fixed vocabulary of K words. An embedding matrix $E$ is used to narrow its representation. The embedding weights $E$ are learnt end-to-end by the model as well.
    * The decoder LSTM uses a deep layer between $h_t$ and $y_t$. It is called a deep output layer and is described in [section 3.2.2 of this paper](https://www.semanticscholar.org/paper/How-to-Construct-Deep-Recurrent-Neural-Networks-Pascanu-G%C3%BCl%C3%A7ehre/533ee188324b833e059cb59b654e6160776d5812). That is:
    $$ p(y_t) = Softmax \Big( f_out(Ey_{t-1}, h_t, \hat{z}_t) \Big) $$
* Initialization MLPs: Two MLPs are used to produce the initial memory-state of the LSTM as well as $h_{t-1}$ value. Each MLP takes in the entire image's features (i.e. average of $a_i$) as its input and is trained end-to-end.
    $$ c_o = f_{init,c}\Big( \sum_i^L a_i \Big) $$
    $$ h_o = f_{init,h}\Big( \sum_i^L a_i \Big) $$
* Training:
    * 3 models from above - all except the conv-net - are trained end-to-end using SGD
    * The model is trained for a variable number of time steps - depending on each batch

## References
1. Show, Attend and Tell
    * [Paper](https://www.semanticscholar.org/paper/Show-Attend-and-Tell-Neural-Image-Caption-Generati-Xu-Ba/146f6f6ed688c905fb6e346ad02332efd5464616)
    * [Slides](https://pdfs.semanticscholar.org/b336/f6215c3c15802ca5327cd7cc1747bd83588c.pdf?_ga=2.52116077.559595598.1498604153-2037060338.1496182671)
    * [Author's Theano code](https://github.com/kelvinxu/arctic-captions)
1. [Simonyan, Karen and Andrew Zisserman. “Very Deep Convolutional Networks for Large-Scale Image Recognition.” CoRR abs/1409.1556 (2014): n. pag.](http://www.robots.ox.ac.uk/~vgg/research/very_deep/)
1. [im2latex solution of Harvard NLP](http://lstm.seas.harvard.edu/latex/)
1. [im2latex-dataset tools forked from Harvard NLP](https://github.com/untrix/im2latex-dataset)

In [16]:
import pandas as pd
import os
from six.moves import cPickle as pickle
import dl_commons as dlc
import tf_commons as tfc
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Embedding, Dense, Activation, Dropout, Concatenate, Permute
from keras.callbacks import LambdaCallback
from keras.models import Model
from keras import backend as K
from keras.engine import Layer
import keras
import threading
import numpy as np
import collections
from Im2LatexDecoderRNNParams import D_RNN
from Im2LatexDecoderRNN import Im2LatexDecoderRNN
from Im2LatexModel import Im2LatexModel, HYPER

# TODOs
* Implement the beta scalar ('selector') that scales alpha.

In [2]:
data_folder = '../data/generated2'

### HyperParams

In [3]:
def get_vocab_size(data_dir_):
    df_vocab = pd.read_pickle(os.path.join(data_folder, 'df_vocab.pkl'))
    return df_vocab.id.max() + 1

### Encoder Model
[VGG ConvNet (16 or 19)](http://www.robots.ox.ac.uk/~vgg/research/very_deep/) without the top-3 layers
* Pre-initialized with the VGG weights but allowed to train
* The ConvNet outputs $D$ dimensional vectors in a WxH grid where W and H are scaled-down dimensions of the input image size (due to 5 max-pool layers). Defining $W.H \equiv L$ the ConvNet output represents L locations of the image $i \in [1,L]$ and correspondingly outputs to L annotation vectors $a_i$, each of size $D$.

The conv-net is *not trained* in the original paper and therefore the files can be separately preprocessed and their outputs directly fed into the model.

### Input Generator

In [4]:
def make_batch_list(df_, batch_size_):
    ## Make a list of batches
    bin_lens = sorted(df_.bin_len.unique())
    bin_counts = [df_[df_.bin_len==l].shape[0] for l in bin_lens]
    batch_list = []
    for i in range(len(bin_lens)):
        bin_ = bin_lens[i]
        num_batches = (bin_counts[i] // batch_size_)
        ## Just making sure bin size is integral multiple of batch_size.
        ## This is not a requirement for this function to operate, rather
        ## is a way of possibly catching data-corrupting bugs
        assert (bin_counts[i] % batch_size_) == 0
        batch_list.extend([(bin_, j) for j in range(num_batches)])

    np.random.shuffle(batch_list)
    return batch_list

class ShuffleIterator(object):
    def __init__(self, df_, batch_size_):
        self._df = df_.sample(frac=1)
        self._batch_size = batch_size_
        self._batch_list = make_batch_list(self._df, batch_size_)
        self._next_pos = 0
        self._num_items = (df_.shape[0] // batch_size_)
        self.lock = threading.Lock()
        
#     def __iter__(self):
#         return self
    
    def next(self):
        ## This is an infinite iterator
        with self.lock:
            if self._next_pos >= self._num_items:
                ## Shuffle the samples
                self._df = self._df.sample(frac=1)
                ## Reshuffle the bin/batch-list
                np.random.shuffle(self._batch_list)
#                 self._batch_list = make_batch_list(self._df, batch_size_)
                self._next_pos = 0
            next_pos = self._next_pos
            self._next_pos += 1
        
        batch = self._batch_list[next_pos]
        df_bin = self._df[self._df.bin_len == batch[0]]
        assert df_bin.bin_len.iloc[batch[1]*self._batch_size] == batch[0]
        assert df_bin.bin_len.iloc[(batch[1]+1)*self._batch_size-1] == batch[0]
        return df_bin.iloc[batch[1]*self._batch_size : (batch[1]+1)*self._batch_size]

class BatchIterator(ShuffleIterator):
    def __init__(self, raw_data_dir_, image_dir_):
        self._padded_im_dim = padded_image_dim_
        self._image_dir = image_dir_
        self._seq_data = pd.read_pickle(os.path.join(raw_data_dir_, 'raw_seq_train.pkl'))
        df = pd.read_pickle(os.path.join(raw_data_dir_, 'df_train.pkl'))
        batch_size = pd.read_pickle(os.path.join(raw_data_dir_, 'batch_size.pkl'))
        Shuffler.__init__(self, df, batch_size)

    @staticmethod
    def get_image_matrix(image_file_, height_, width_, padded_height_, padded_width_):
        MAX_PIXEL = 255.0 # Make sure this is a float literal
        ## Load image and convert to a 3-channel array
        im_ar = ndimage.imread(os.path.join(image_dir_, image_file_), mode='RGB')
        ## normalize values to lie between -1.0 and 1.0.
        ## This is done in place of data whitening - i.e. normalizing to mean=0 and std-dev=0.5
        ## Is is a very rough technique but legit for images
        im_ar = (im_ar - MAX_PIXEL/2.0) / MAX_PIXEL
        height, width, channels = im_ar.shape
        assert height == height_
        assert width == width_
        assert channels == 3
        if (height < padded_height_) or (width < padded_width_):
            ar = np.full((padded_height_, padded_width_), 0.5, dtype=np.float32)
            h = (padded_height_-height)//2
            ar[h:h+height, 0:width] = im_ar
            im_ar = ar

        return im_ar

    def next(self):
        df_batch = Shuffler.next(self)[['image', 'height', 'width', 'bin_len']]
        im_batch = [
            self._get_image_array(os.path.join(self._image_dir, row[0]), row[1], row[2], 
                                  self.padded_im_dim.height, self.padded_im_dim.width)
            for row in df_batch.image.itertuples()
        ]   
        im_batch = np.asarray(im_batch)
        
        bin_len = df_batch.bin_len.iloc[0]
        seq_batch = self._seq_data[bin_len][df_batch.index].values
        return dlc.Properties({'im':im_batch, 
                               'seq':seq_batch})

# class FormulaIterator(ShuffleIterator):
#     def __init__(self, df_, batch_size_, data_dir_, seq_filename_):
#         Shuffler.__init__(self, df_, batch_size_)
#         self._seq_data = pd.read_pickle(os.path.join(data_dir_, seq_filename_))
        
#     def next(self):
#         df_batch = Shuffler.next(self)['bin_len']
#         bin_len = df_batch.iloc[0].bin_len
#         return self._seq_data[bin_len][df_batch.index].values

In [11]:
df_train = pd.read_pickle(os.path.join(data_folder, 'training', 'df_train.pkl'))
df_train

,image,height,width,word2id_len,bin_len,word2id,latex_ascii,padded_seq,padded_seq_len,seq_len
1,868d5037af9e4b4_basic.png,94,962,125,151,"[529, 544, 523, 552, 18, 554, 1, 29, 1, 8, 17,...",ds^{2} = (1 - {qcos\theta\over r})^{2\over 1 +...,"[529, 544, 523, 552, 18, 554, 1, 29, 1, 8, 17,...",151,126
2,af0b6c3ee18804a_basic.png,87,291,47,51,"[517, 231, 524, 552, 410, 1, 533, 540, 541, 53...",\widetilde\gamma_{\rm hopf}\simeq\sum_{n>0}\wi...,"[517, 231, 524, 552, 410, 1, 533, 540, 541, 53...",51,48
3,dda45eca6d32fa3_basic.png,35,405,53,61,"[8, 552, 159, 1, 44, 554, 524, 526, 1, 532, 9,...","({\cal L}_a g)_{ij} = 0, \ \ \ \ ({\cal L}_a H...","[8, 552, 159, 1, 44, 554, 524, 526, 1, 532, 9,...",61,54
4,67eb249ed1c20d2_basic.png,60,521,69,71,"[51, 524, 552, 544, 545, 526, 545, 554, 1, 29,...",S_{stat} = 2\pi \sqrt{N_5^{(1)} N_5^{(2)} N_5^...,"[51, 524, 552, 544, 545, 526, 545, 554, 1, 29,...",71,70
5,89ef1bacdfcca24_basic.png,102,206,40,51,"[238, 1, 46, 524, 19, 1, 29, 1, 454, 441, 1, 5...",\hat N_3 = \sum\sp f_{j=1}a_j\sp {\dagger} a_j...,"[238, 1, 46, 524, 19, 1, 29, 1, 454, 441, 1, 5...",51,41
7,6135540f1af5ff3_basic.png,30,282,42,51,"[60, 12, 523, 552, 10, 554, 529, 60, 12, 523, ...","\,^{*}d\,^{*}H=\kappa \,^{*}d\phi = J_B . \lab...","[60, 12, 523, 552, 10, 554, 529, 60, 12, 523, ...",51,43
8,92656e604bed774_basic.png,80,817,93,111,"[552, 376, 7, 7, 364, 1, 33, 554, 1, 11, 552, ...",{\phi''\over A} +{1\over A}\left( -{1\over 2}{...,"[552, 376, 7, 7, 364, 1, 33, 554, 1, 11, 552, ...",111,94
9,892756cc4445c69_basic.png,35,296,37,41,"[267, 552, 538, 526, 549, 548, 554, 373, 524, ...",\label{maxw}\partial_{\mu} (F^{\mu\nu}-ej^{\mu...,"[267, 552, 538, 526, 549, 548, 554, 373, 524, ...",41,38
10,d9934420d849263_basic.png,75,697,95,111,"[37, 524, 552, 33, 36, 45, 554, 1, 29, 1, 228,...",E_{ADM} = \frac{1}{16 \pi G_{10}} \oint_{\inft...,"[37, 524, 552, 33, 36, 45, 554, 1, 29, 1, 228,...",111,96
11,93719dbe3fdbba9_basic.png,81,698,72,81,"[267, 552, 531, 534, 530, 543, 530, 541, 554, ...",\label{fierep}P_{(2)}^-=\int \beta d\beta d^9p...,"[267, 552, 531, 534, 530, 543, 530, 541, 554, ...",81,73


In [12]:
df_train.shape

(87552, 10)

In [13]:
raw_seq_train = pd.read_pickle(os.path.join(data_folder, 'training', 'raw_seq_train.pkl'))
raw_seq_train.keys()

[71, 41, 111, 81, 51, 151, 91, 61, 31]

In [18]:
with open(os.path.join(data_folder, 'training', 'padded_image_dim.pkl'), 'rb') as f:
  padded_image_dim = pickle.load(f)
print(padded_image_dim)

{'width': 1075, 'height': 120}


#### Decoder Model
A dense (FC) attention model: The deterministic soft-attention model of the paper computes $\alpha_{t,i}$ which is used to select or blend the $a_i$ vectors before being fed as inputs to the decoder LSTM network (see below).
* Inputs to the attention model are $a_i$ and $h_{t-1}$ (previous hidden state of LSTM network - see below) and $$\alpha_{t,i} = softmax ( f_{att}(a_i, h_{t-1}) )$$
* Note that the model $f_{att}$ shares weights across all values of a_i (i.e. for all i = 1-L). Therefore the shared weight matrix for all a_i has shape (D, D), while shape of a is (B, L, D) where is B=batch-size. Weight matrix of h_i is separate and has the expected shape (n, D). This sharing of weights across a_i is interesting.

A Decoder model: A conditioned LSTM that outputs probabilities of the text tokens $y_t$ at each step. The LSTM is conditioned upon $z_t = \sum_i^L(\alpha_{t,i}.a_i)$ and takes the previous hidden state $h_{t-1}$ as input. In addition, an embedding of the previous output $Ey_{t-1}$ is also input to the LSTM. At training time, $y_{t-1}$ would be derived from the training samples, while at inferencing time it would be fed-back from the previous predicted word.
* $y$ is taken from a fixed vocabulary of K words. An embedding matrix $E$ is used to narrow its representation to an $m$ dimensional dense vector. The embedding weights $E$ are learnt end-to-end by the model as well.
* The decoder LSTM uses a deep layer between $h_t$ and $y_t$. It is called a deep output layer and is described in [section 3.2.2 of this paper](https://www.semanticscholar.org/paper/How-to-Construct-Deep-Recurrent-Neural-Networks-Pascanu-G%C3%BCl%C3%A7ehre/533ee188324b833e059cb59b654e6160776d5812). That is:
$$ p(y_t) = Softmax \Big( f_out(Ey_{t-1}, h_t, \hat{z}_t) \Big) $$
* Optionally $z_t = \beta \sum_i^L(\alpha_{t,i}.a_i)$ where $\beta = \sigma(f_{\beta}(h_{t-1}))$ is a scalar used to modulate the strength of the context. It turns out that for the original use-case of caption generation, the network would learn to emphasize objects by turning up the value of this scalar when it was focusing on objects. It is not clear at this time whether we'll need this feature for im2latex.


In [5]:
# with tf.variable_scope('test_1'):
#     m = Im2LatexModel().build()
#     print 'yProbs shape = ', K.int_shape(m.yProbs)

In [6]:
def test_rnn():
    B = HYPER.B
    Kv = HYPER.K
    L = HYPER.L

    ## TODO: Introduce Stochastic Learning

    m = Im2LatexModel()
    rv = dlc.Properties()
    im = tf.placeholder(dtype=tf.float32, shape=(HYPER.B,) + HYPER.image_shape, name='image_batch')
    y_s = tf.placeholder(tf.int32, shape=(HYPER.B, None))
    print 'y_s[:,0] shape: ', K.int_shape(y_s[:,0])
    print 'embedding_lookup shape: ', K.int_shape(m._embedding_lookup(y_s[:,0]))
    a = m._build_image_context(im)
    rnn = Im2LatexDecoderRNN(D_RNN, a, 10)
    print 'rnn ', rnn.state_size, rnn.output_size
    #init_c, init_h = m._build_init_layer(a)

    decoder = tf.contrib.seq2seq.BeamSearchDecoder(rnn, 
                                                   m._embedding_lookup,
                                                   y_s[:,0],
                                                   0,
                                                   rnn.zero_state(D_RNN.B*rnn.BeamWidth, tf.float32),
                                                   beam_width=rnn.BeamWidth)
    
    print 'decoder._start_tokens: ', K.int_shape(decoder._start_tokens)
    print 'decoder._start_inputs: ', K.int_shape(decoder._start_inputs)
    final_outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(decoder,
                                                                                           maximum_iterations=D_RNN.Max_Seq_Len + 10,
                                                                                           swap_memory=True)
    print 'final_outputs: ', K.int_shape(final_outputs.predicted_ids)
    print 'final_state: ', (final_state)
    print 'final_sequence_lengths', (final_sequence_lengths)
    
    return final_outputs.predicted_ids

def visualize_rnn():
    graph = tf.Graph()
    with graph.as_default():

        a = tf.placeholder(dtype=tf.float32, shape=(D_RNN.B, D_RNN.L, D_RNN.D), name='image_a')
        init_Ex = tf.placeholder(tf.float32, shape=(D_RNN.B, D_RNN.m), name='init_Ex')
        rnn = Im2LatexDecoderRNN(D_RNN, a, 1)
        init_state = rnn.zero_state(D_RNN.B*rnn.BeamWidth, tf.float32)
        print 'type of init_state = ', type(init_state)
        output, state = rnn(init_Ex, init_state)
        
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:

            print 'Flushing graph to disk'
            tf_sw = tf.summary.FileWriter(tfc.makeTBDir(D_RNN.tb), graph=graph)
            tf_sw.flush()

visualize_rnn()


AssertionError: 

In [ ]:
# ## Conv-net
# # K.set_image_data_format('channels_last')
# #image_input = Input(shape=HYPER.image_shape, name='image_input')
# image_input = tf.placeholder(dtype=tf.float32, shape=(HYPER.B,) + HYPER.image_shape, name='image_batch2')
# convnet = VGG16(include_top=False, weights='imagenet', pooling=None, input_shape=HYPER.image_shape)
# convnet.trainable = False
# print 'convnet output_shape = ', convnet.output_shape
# a = convnet(image_input)
# a

# End